# **Ejercicio 2 (Maximización de Volumen con PSO)**

**Objetivo:** Determinar las dimensiones de una caja con base cuadrada que maximicen su volumen, utilizando una cantidad fija de 10 m² de material y aplicando el algoritmo de Optimización por Enjambre de Partículas (PSO).

## **Configuración Inicial**

### **Instalar y Cargar Librerías**
Se importan las librerías necesarias. DEAP es el framework principal para implementar el algoritmo PSO.

In [14]:
# Si es necesario, instalar el paquete DEAP
# !pip install deap

In [15]:
# Importar paquetes
import matplotlib.pyplot as plt
from deap import base, creator, tools
import pandas as pd
import operator
import random
import numpy
import math

## **Planteamiento del Problema**

Se desea construir una caja con base cuadrada y sin tapa superior. Se dispone de 10 m² de material para su construcción.

* **Variables:**
    * `x`: Lado de la base cuadrada.
    * `h`: Altura de la caja.

* **Función Objetivo (Volumen):**

    $$ V = \text{Área de la base} \times \text{Altura} = x^2 \cdot h $$

* **Restricción (Área de Superficie):**
El material total es la suma del área de la base y las cuatro caras laterales.

    $$ \text{Área Total} = x^2 + 4xh = 10 $$

    De esta restricción, podemos despejar la altura `h` en función de `x`:

    $$ h = \frac{10 - x^2}{4x} $$

    Sustituyendo `h` en la función de volumen, obtenemos la función objetivo dependiente de una sola variable, `x`:

    $$ \text{Maximizar } V(x) = x^2 \left( \frac{10 - x^2}{4x} \right) = \frac{10x - x^3}{4} $$

* **Restricciones de Dominio:**
    * `x > 0` (El lado debe ser positivo).
    * La restricción `h > 0` (la altura debe ser positiva) nos lleva a la siguiente lógica para encontrar el límite superior de `x`:

        1.  **Fórmula del Área:** El área total del material es la suma de la base y los cuatro lados.
            $$ x^2 + 4xh = 10 $$

        2.  **Despejar la altura (h):** Expresamos la altura en función de `x`.
            $$ h = \frac{10 - x^2}{4x} $$

        3.  **Aplicar la Restricción:** Como la altura debe ser mayor que cero (`h > 0`), y ya que `x` (una longitud) también debe ser positivo, el numerador de la fracción tiene que ser positivo.
            $$ 10 - x^2 > 0 $$

        4.  **Resolver la Desigualdad:**
            $$ 10 > x^2 $$
            $$ \sqrt{10} > x $$

        5.  **Conclusión:**
            $$ x < \sqrt{10} \approx 3.16 $$

    Por lo tanto, el lado `x` de la base debe ser menor que aproximadamente 3.16 metros para que la caja tenga una altura positiva.

### **2. Funciones del Problema**
* **`objective_function`**: Calcula el volumen de la caja, que es el valor a maximizar.
* **`feasible`**: Verifica si una partícula (un valor de `x`) cumple con las restricciones de dominio.

In [16]:
# Función Objetivo
# Maximizar el volumen V(x) = (10x - x³) / 4
def objective_function(individual):
    x = individual[0]
    volume = (10 * x - (x**3)) / 4
    return volume,

In [17]:
# Restricción del Problema
# Verifica si la partícula es una solución factible (0 < x < sqrt(10)).
def feasible(individual):
    x = individual[0]
    if x < 0:
        return False
    # La restricción h > 0 implica x < sqrt(10)
    if x > math.sqrt(10):
      return False
    return True

## **Configuración del Algoritmo PSO**

### **1. Creación de Tipos con `creator`**
Se definen las estructuras para el **Fitness** y la **Partícula**.

* **`FitnessMax`**: Define un objetivo de maximización con `weights=(1.0,)`.
* **`Particle`**: Define la estructura de una partícula con sus atributos específicos para PSO: `speed` (velocidad), `smin`/`smax` (límites de velocidad) y `best` (mejor posición personal).

In [18]:
# Crear los tipos de Fitness y Partícula
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, smin=None, smax=None, best=None)

/home/santiago/anaconda3/envs/MADSI/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/santiago/anaconda3/envs/MADSI/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Particle' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


### **2. Funciones para Generar y Actualizar Partículas**
* **`generate`**: Inicializa una partícula con una posición y velocidad aleatorias.
* **`updateParticle`**: Implementa la lógica de movimiento de PSO, actualizando la velocidad y posición de la partícula basándose en su mejor posición y la del enjambre.

In [19]:
# Función para generar una partícula con posición y velocidad iniciales aleatorias.
def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size))
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

In [20]:
# Función para actualizar la velocidad y posición de una partícula.
def updateParticle(part, best, phi1, phi2):
    # Componente cognitivo (hacia la mejor posición personal)
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    # Componente social (hacia la mejor posición global)
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    # Actualizar velocidad y aplicar límites
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if abs(speed) < part.smin:
            part.speed[i] = math.copysign(part.smin, speed)
        elif abs(speed) > part.smax:
            part.speed[i] = math.copysign(part.smax, speed)
    # Actualizar posición
    part[:] = list(map(operator.add, part, part.speed))

### **3. Creación de la `Toolbox`**
Se registran las funciones y operadores en la `Toolbox` para su uso en el algoritmo.

In [21]:
# Crear la caja de herramientas (Toolbox)
toolbox = base.Toolbox()

# Registrar la función para generar partículas.
# size=1 (solo la variable x), pmin/pmax (límites de posición), smin/smax (límites de velocidad).
toolbox.register("particle", generate, size=1, pmin=0, pmax=5, smin=-0.5, smax=0.5)

# Registrar la función para generar la población.
toolbox.register("population", tools.initRepeat, list, toolbox.particle)

# Registrar la función para actualizar las partículas.
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)

# Registrar la función de evaluación con penalización para soluciones no factibles.
toolbox.register("evaluate", objective_function)
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, -100000)) # Penalización alta si no es factible

### **4. Definición de Parámetros y Estadísticas**
Se configuran los parámetros del algoritmo y el registro de estadísticas.

In [22]:
# Parámetros del algoritmo PSO
initial_population = 100    # Número de partículas
num_ite = 100               # Número de iteraciones
best = None                 # Mejor partícula global

In [23]:
# Creación de la población inicial
pop = toolbox.population(n=initial_population)

In [24]:
# Configuración de las estadísticas a monitorear
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

In [25]:
# Configuración del Logbook para guardar el historial
logbook = tools.Logbook()
logbook.header = ["gen", "evals"] + stats.fields

## **Ejecución del Algoritmo PSO**
Se ejecuta el bucle principal de optimización.

In [26]:
# Bucle principal de la optimización PSO
for iteration in range(num_ite):
    for part in pop:
        # Evaluar fitness
        part.fitness.values = toolbox.evaluate(part)
        # Actualizar mejor posición personal (pbest)
        if not part.best or part.best.fitness < part.fitness:
            part.best = creator.Particle(part)
            part.best.fitness.values = part.fitness.values
        # Actualizar mejor posición global (gbest)
        if not best or best.fitness < part.fitness:
            best = creator.Particle(part)
            best.fitness.values = part.fitness.values
    # Actualizar velocidad y posición de todas las partículas
    for part in pop:
        toolbox.update(part, best)

    # Guardar y mostrar las estadísticas de la iteración
    logbook.record(gen=iteration, evals=len(pop), **stats.compile(pop))
    print(logbook.stream)

gen	evals	avg     	std    	min    	max    
0  	100  	-37998.6	48539.7	-100000	3.04039
1  	100  	-28998.2	45377.3	-100000	3.04287
2  	100  	-20997.9	40731.9	-100000	3.04271
3  	100  	-14997.7	35708.1	-100000	3.0429 
4  	100  	-6997.45	25515.4	-100000	3.04289
5  	100  	-5997.33	23749.4	-100000	3.04288
6  	100  	-3997.24	19596.5	-100000	3.0429 
7  	100  	-2997.23	17059.2	-100000	3.0429 
8  	100  	-2997.2 	17059.2	-100000	3.04278
9  	100  	-1997.18	14000.4	-100000	3.0429 
10 	100  	-997.149	9950.16	-100000	3.0429 
11 	100  	-997.103	9950.17	-100000	3.0429 
12 	100  	-997.124	9950.16	-100000	3.0429 
13 	100  	-997.139	9950.16	-100000	3.0429 
14 	100  	-997.113	9950.16	-100000	3.0429 
15 	100  	-997.113	9950.16	-100000	3.0429 
16 	100  	2.88281 	0.225477	1.39973	3.0429 
17 	100  	2.91577 	0.131587	2.50243	3.0429 
18 	100  	2.90797 	0.162706	2.11789	3.0429 
19 	100  	2.91109 	0.136141	2.4839 	3.0429 
20 	100  	2.92688 	0.134044	2.3188 	3.04276
21 	100  	2.9047  	0.169439	1.78237	3.0429 
22 	1

## **Resultados Finales**
Se muestran las dimensiones y el volumen máximo de la caja encontrados por el algoritmo.

In [27]:
# Imprimir la mejor solución encontrada
x_optimo = best[0]
h_optima = (10 - x_optimo**2) / (4 * x_optimo) if x_optimo != 0 else 0

print(f'Lado de la base (x): {x_optimo:.4f} m')
print(f'Altura (h): {h_optima:.4f} m')

Lado de la base (x): 1.8257 m
Altura (h): 0.9129 m


In [28]:
# Evaluar la mejor solución
print(f'Volumen Máximo: {objective_function(best)[0]:.4f} m³')
print(f'Área de material utilizada: {x_optimo**2 + 4 * x_optimo * h_optima:.4f} m²')

Volumen Máximo: 3.0429 m³
Área de material utilizada: 10.0000 m²


In [29]:
# Verificar si la solución es factible
print(f'Cumple restricciones: {feasible(best)}')

Cumple restricciones: True
